# 使用 DataLad 和 AWS 从 OpenNeuro 下载数据的全面教程

## 概述
OpenNeuro 是一个免费的开放平台，用于共享神经成像数据，支持 MRI、MEG、EEG 等多种格式。本教程将指导您在 Linux、macOS 和 Windows 系统上使用 DataLad 和 AWS 下载 OpenNeuro 数据集。DataLad 是一个基于 Git 和 git-annex 的数据管理工具，能够高效处理大数据集，而 AWS 则提供通过 Amazon S3 访问数据的途径。



## 一、环境准备

### 1.1 系统要求
在开始之前，请确保您的系统满足以下条件：
- 稳定的互联网连接。
- 已安装 Python 3（推荐 3.7 或更高版本）。
- 基本的命令行操作知识。
- 一个 GitHub 账户（可选，用于 SSH 方式访问 DataLad 数据集）。
  


## 二、工具安装指南

### 2.1 DataLad
DataLad 是一个跨平台的工具，需要先安装 Git 和 git-annex，然后再安装 DataLad 本身。

#### 2.1.1 Linux (Ubuntu/Debian)

```bash
# 更新包管理器
sudo apt-get update

# 安装 Git
sudo apt-get install git

# 安装 git-annex
sudo apt-get install git-annex

# 创建并激活虚拟环境（可选但推荐）
python3 -m venv ~/datalad_env
source ~/datalad_env/bin/activate

# 安装 DataLad
pip install datalad
```

#### 2.1.2 macOS（Homebrew）
```bash
# 安装 Homebrew（如果未安装）
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

# 安装 Git 和 git-annex
brew install git
brew install git-annex

# 创建并激活虚拟环境
python3 -m venv ~/datalad_env
source ~/datalad_env/bin/activate

# 安装 DataLad
pip install datalad
```

### 2.2 AWS CLI

- Windows 需要额外步骤，因为 git-annex 和 DataLad 的原生支持有限。因此，更推荐Windows用户使用AWS CLI完成下载。
  
- AWS CLI 用于通过 Amazon S3 下载 OpenNeuro 数据，无需 AWS 账户即可访问公开数据。

#### 2.2.1 Linux (Ubuntu/Debian)和macOS

```bash
# 安装 AWS CLI
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install
# 验证安装
aws --version
```

#### 2.2.2 Windows
```powershell
# 下载安装包
Invoke-WebRequest -Uri "https://awscli.amazonaws.com/AWSCLIV2.msi" -OutFile "$env:TEMP\AWSCLIV2.msi"

# 静默安装
Start-Process -Wait -FilePath "$env:TEMP\AWSCLIV2.msi" -ArgumentList "/quiet"

# 验证安装
aws --version
```

## 三、基于OpenNeuro下载数据
### 3.1 DataLad
```bash
# 基础操作
# 克隆元数据（不下载实际文件）
datalad clone https://github.com/OpenNeuroDatasets/ds002336.git
cd ds002336

# 查看文件夹数据的大小
du -sh

# 按需获取数据
datalad get sub-01/anat/  # 下载特定被试的解剖数据
datalad get .            # 下载完整数据集

# 更新数据集
datalad update --merge
```

```bash
# 其他操作
# 访问数据集特定的版本
git tag

# 切换到某个特定的版本
git checkout 1.0.5
```

### 3.2 AWS CLI

```bash
# 下载完整数据集
aws s3 sync --no-sign-request s3://openneuro.org/ds002336 ./ds002336/

# 选择性下载（示例：只要 BIDS 原始数据）
aws s3 sync --no-sign-request \
    s3://openneuro.org/ds002336 ./ds002336/ \
    --exclude "*" \
    --include "sub-*/ses-*/*" \
    --include "dataset_description.json"

# 断点续传（网络不稳定时）
aws configure set default.s3.max_concurrent_requests 5  # 限制并发数
```